# StyleCLIP+: Towards an Extended and User-Friendly StyleCLIP, <b>Gradio Demo 1</b>
Fall 2023 CS470 Introduction to Artificial Intelligence Project 2 (Implementation), Team 3  
Myeongseok Kwon<sup>1</sup>, Junhak Ha<sup>2</sup>, Dongwan Hong<sup>3</sup>, Kyeongmin Lee<sup>4</sup>, Bryan Nathanael Wijaya<sup>5*</sup>  
<small><sup>1</sup>20170042, <sup>2</sup>20190683, <sup>3</sup>20190696, <sup>4</sup>20200429, <sup>5</sup>20200735, <sup>*</sup>Team Leader</small>

## Ablation Study of LLM Usage for Text Prompt Refinement

In [1]:
#@title Setting up Gradio for demo
!pip install kaleido cohere openai tiktoken gradio -q
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.3 MB/s

In [2]:
#@title Environment setup
import time, os
import torch
from torchvision.utils import make_grid
from torchvision.transforms import ToPILImage
if not os.path.exists('./StyleCLIP'):
    !git clone https://github.com/bramyeon/StyleCLIP.git
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
from tqdm import tqdm

# Development environment setup for latent optimization
os.chdir(f'./StyleCLIP')
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# downloads StyleGAN's weights and facial recognition network weights
from StyleCLIP.utils import google_drive_paths, llm_setup, prompt_refiner
ckpt = 'stylegan2-ffhq-config-f.pt'
ckpt_id = google_drive_paths[ckpt].split('=')[-1]
ids = [ckpt_id, '1N0MZSqPRJpLfP4mFQCS14ikrVSe8vQlL']
for file_id in ids:
    downloaded = drive.CreateFile({'id':file_id})
    downloaded.FetchMetadata(fetch_all=True)
    downloaded.GetContentFile(downloaded.metadata['title'])
llm = llm_setup()
from StyleCLIP.optimization.run_optimization import main
from argparse import Namespace

Cloning into 'StyleCLIP'...
remote: Enumerating objects: 925, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 925 (delta 188), reused 206 (delta 137), pack-reused 614
Receiving objects: 100% (925/925), 200.06 MiB | 24.65 MiB/s, done.
Resolving deltas: 100% (332/332), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.0 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-qn1igzf4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-qn1igzf4
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=41675f6a13b625f6426988b43ef540d707566a10c3461bc08d2b46c72839d58a
  Stored in directory: /tmp/pip-ephem-wheel-cache-68x_6wck/wheels/da/2b/4c/d6691fa9597aac8bb85d2a

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [3]:
#@title Demo 1 Gradio

def demo1_fn(seed, text_prompt, optimization_steps, l2_lambda, id_lambda):
    model = "StyleGAN2, FFHQ, human face, stylegan2-ffhq-config-f.pt"
    architecture, dataset, description, ckpt = model.split(', ')

    # setting up the arguments for Latent Optimization
    experiment_type = 'edit'
    latent_path = None
    stylespace = False
    create_video = False

    args = {
        "description": text_prompt,
        "ckpt": ckpt,
        "stylegan_size": 1024,
        "lr_rampup": 0.05,
        "lr": 0.1,
        "step": optimization_steps,
        "mode": experiment_type,
        "l2_lambda": l2_lambda,
        "id_lambda": id_lambda,
        "mse_lambda": 0,
        'work_in_stylespace': stylespace,
        "latent_path": latent_path,
        "truncation": 0.7,
        "save_intermediate_image_every": 1 if create_video else 20,
        "results_dir": "results",
        "ir_se50_weights": "model_ir_se50.pth"
    }

    torch.manual_seed(seed)
    result = main(Namespace(**args))
    result_image = ToPILImage()(make_grid(result.detach().cpu(), normalize=True, scale_each=True, padding=0)) #range=(-1, 1)
    h, w = result_image.size
    result_image.resize((h // 2, w // 2))

    # text prompt refined version
    text_prompt = prompt_refiner(llm, text_prompt, description)
    args['description'] = text_prompt
    torch.manual_seed(seed)
    result_llm = main(Namespace(**args))
    result_image_llm = ToPILImage()(make_grid(result_llm.detach().cpu(), normalize=True, scale_each=True, padding=0)) #range=(-1, 1)
    h, w = result_image_llm.size
    result_image_llm.resize((h // 2, w // 2))

    return result_image, args['description'], result_image_llm

demo1 = gr.Interface(
    fn=demo1_fn, title="CS470 Team 3 LLM Demo",
    inputs=[gr.Slider(0, 1000, step=1, value=470, label="Seed for original FACE image"), \
            gr.Textbox(lines=1, placeholder="e.g., a surprised book", label="Target prompt"), \
            gr.Slider(0, 500, step=1, value=40, label="Optimization steps"), \
            gr.Slider(0, 0.1, step=0.001, value=0.008, label="L2-loss strength"), \
            gr.Slider(0, 0.1, step=0.001, value=0.005, label="ID-loss strength")],
    outputs=[gr.Image(label="Original vs. target image (w/o LLM refinement)"), \
             gr.Textbox(label="LLM-refined target prompt"), \
             gr.Image(label="Original vs. target image (with LLM refinement)")])

if __name__ == "__main__":
    demo1.launch(show_api=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://eeb2560007890d3a19.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
